In [1]:
import time
from config import *
import math
from datetime import date
from math_utils import *
from network_utils import *
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
filepath = "data/GLP/"

In [3]:
import time
import datetime

timestamp = int(time.mktime(datetime.datetime.strptime("17/07/2023", "%d/%m/%Y").timetuple()))

block_number = get_block_by_timestamp_arb(int(timestamp))

In [4]:
block_number

111858291

In [5]:
with open("GLP-28MARCH2024.json", 'r') as f:
  pool = json.load(f)

pool

{'name': 'SY GLP',
 'SY': '0x2066a650AF4b6895f72E618587Aad5e8120B7790',
 'YT': '0x56051f8e46b67b4d286454995dbc6f5f3c433e34',
 'PT': '0x96015d0fb97139567a9ba675951816a0bb719e3c',
 'market': '0x7D49E5Adc0EAAD9C027857767638613253eF125f',
 'expiry': 1711584000,
 'scalarRoot': '10484880000000000000',
 'initAnchor': '1285295000000000000'}

In [6]:
YT_contract = get_contract_arb(address=pool["YT"])
index = to_normal_float(YT_contract.functions.pyIndexStored().call(block_identifier=block_number))
print(index)

time.sleep(5)
market_contract = get_contract_arb(address=pool["market"])

time.sleep(5)
SY_contract = get_contract_arb(address=pool["SY"])

1.0


In [7]:
timestamp = int(time.mktime(datetime.datetime.strptime("16/07/2023", "%d/%m/%Y").timetuple()))
data_onchain = get_info(SY_contract, YT_contract, market_contract, timestamp, 'arb')
data_onchain


{'date': '16-07-23',
 'asset': 14923635.628499085,
 'PT': 5385206.212491589,
 'LP': 10436069.503493262,
 'SY': 14923635.628499085,
 'exchange_rate': 1.1103840174259973,
 'X_reward_index': 4.1199165529132e-05,
 'PENDLE_reward_index': 0.5914651399936859,
 'average_boost_factor': 1.7556146574750757,
 'SY_index': 1.0}

In [8]:
import pandas as pd

data_df = pd.DataFrame()

timestamp_start = int(time.mktime(datetime.datetime.strptime("24/04/2023", "%d/%m/%Y").timetuple()))
timestamp_end = int(time.mktime(datetime.datetime.strptime("24/07/2023", "%d/%m/%Y").timetuple()))

for i in range(timestamp_start, timestamp_end, DAY_DELTA * DAY):
    data_onchain = pd.DataFrame(get_info(SY_contract, YT_contract, market_contract, i, 'arb'), index=[0])
    data_df = pd.concat([data_df, data_onchain], ignore_index=True)

In [9]:
IL_A1 = IL(data_df, 'A', False, GLP, ETH)
IL_A2 = IL(data_df, 'A', True, GLP, ETH)
IL_B1 = IL(data_df, 'B', False, GLP, ETH)
IL_B2 = IL(data_df, 'B', True, GLP, ETH)
IL_C1 = IL(data_df, 'C', False, GLP, ETH)
IL_C2 = IL(data_df, 'C', True, GLP, ETH)

In [10]:
write_array_to_csv(IL_A1, filepath + "IL_A1.csv", timestamp_start)
write_array_to_csv(IL_A2, filepath + "IL_A2.csv", timestamp_start)
write_array_to_csv(IL_B1, filepath + "IL_B1.csv", timestamp_start)
write_array_to_csv(IL_B2, filepath + "IL_B2.csv", timestamp_start)
write_array_to_csv(IL_C1, filepath + "IL_C1.csv", timestamp_start)
write_array_to_csv(IL_C2, filepath + "IL_C2.csv", timestamp_start)

In [11]:
LP_profit_PENDLE = in_pool_value(data_df, asset_price=GLP, X_price=ETH, PENDLE_incentive=True, with_YT = False)
LP_profit = in_pool_value(data_df, asset_price=GLP, X_price=ETH, PENDLE_incentive=False, with_YT = False)
LP_profit_PENDLE_YT = in_pool_value(data_df, asset_price=GLP, X_price=ETH, PENDLE_incentive=True, with_YT = True)
LP_profit_YT = in_pool_value(data_df, asset_price=GLP, X_price=ETH, PENDLE_incentive=False, with_YT = True)

[1.0, 1.0071500274440424, 1.0152048469520505, 1.0208391064519255, 1.029284097887465, 1.0412082402741534, 1.0485580797339984, 1.0552938180887315, 1.0645027543253291, 1.0696107547424134, 1.07760488437983, 1.0980940306247182, 1.113251185251469, 1.1224001404101696, 1.1256573941164993, 1.1289216277240866, 1.1440093492463366, 1.1592152365887838, 1.166978013235567, 1.1834968606045304, 1.1883806936950199, 1.1899443253841786, 1.2023305985600543]
[<NA>, 1.0, 1.008055033708723, 1.0136691973628307, 1.022067434380448, 1.0339939419581563, 1.0413174807903902, 1.0480402653223138, 1.0573010696946648, 1.062389132420629, 1.0703286925464846, 1.0894065606589727, 1.1037279869294108, 1.1126770314821905, 1.1156300169370914, 1.1189604394377475, 1.1340217598916407, 1.149267672270303, 1.1569977814883707, 1.1731249303479276, 1.1780918670097602, 1.1796490035523985, 1.1917778029109312]
[<NA>, <NA>, 1.0, 1.0055973381408208, 1.0139408856716527, 1.0258582227128197, 1.0331454598339687, 1.039848004751214, 1.049156099060

In [12]:
write_array_to_csv(LP_profit_PENDLE, "data\LP_profit\GLP_PENDLE.csv", timestamp_start)
write_array_to_csv(LP_profit, "data\LP_profit\GLP.csv", timestamp_start)
write_array_to_csv(LP_profit_PENDLE_YT, "data\LP_profit\GLP_PENDLE_YT.csv", timestamp_start)
write_array_to_csv(LP_profit_YT, "data\LP_profit\GLP_YT.csv", timestamp_start)
